In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.tree as tree
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import decimal
import sklearn.datasets

In [2]:
#https://www.techbeamers.com/python-float-range/#using-yield-to-generate-a-float-range
def float_range(start, stop, step):
  while start < stop:
    yield float(start)
    start += decimal.Decimal(step)


In [3]:
X_bc, y_bc = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train_bc, X_test_bc, y_train_bc, y_test_bc = train_test_split(X_bc, y_bc, random_state=0, test_size=0.4)

df = pd.read_csv("occupancy_data.csv")
#df = sqlContext.read.csv("/FileStore/tables/occ.csv", header=True).toPandas()
X_occ = df.iloc[:,:5]
y_occ = df.iloc[:,5]
X_train_occ, X_test_occ, y_train_occ, y_test_occ = train_test_split(X_occ, y_occ, random_state=0, test_size=0.4)



In [4]:
dt_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for ccp_alpha in list(float_range(0, 0.4, '0.005')):
    clf = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=ccp_alpha, max_depth=5)
    dt_training_pd["bc_alpha"].append(ccp_alpha)
    dt_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    dt_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for ccp_alpha in list(float_range(0, 0.4, '0.005')):
    clf = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=ccp_alpha, max_depth=5)
    dt_training_pd["occ_alpha"].append(ccp_alpha)
    dt_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    dt_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=0.025, max_depth=5)
clf_occ = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=0.005, max_depth=5)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(dt_training_pd))
print(final_df)


bc_alpha bc_training bc_test occ_alpha occ_training occ_test
0 0.000 0.909165 0.929952 0.000 0.994063 0.986802
1 0.005 0.915047 0.929952 0.005 0.988946 0.985880
2 0.010 0.920844 0.938744 0.010 0.988946 0.985880
3 0.015 0.929582 0.956135 0.015 0.988946 0.985880
4 0.020 0.923700 0.956135 0.020 0.988946 0.985880
.. ... ... ... ... ... ...
76 0.380 0.621697 0.689372 0.380 0.792631 0.780233
77 0.385 0.621697 0.636039 0.385 0.792631 0.780233
78 0.390 0.621697 0.636039 0.390 0.792631 0.780233
79 0.395 0.621697 0.636039 0.395 0.792631 0.780233
80 0.400 0.621697 0.636039 0.400 0.792631 0.780233

[81 rows x 6 columns]
 percentage bc_training bc_test occ_train occ_test
0 0.1 1.000000 0.826598 0.997436 0.988127
1 0.2 0.977778 0.920801 0.993086 0.988946
2 0.3 0.982716 0.909207 0.994027 0.988946
3 0.4 0.957407 0.917860 0.991811 0.988946
4 0.5 0.967647 0.915047 0.992221 0.988946
5 0.6 0.953374 0.923743 0.991894 0.988946
6 0.7 0.957895 0.923743 0.991005 0.988946
7 0.8 0.947465 0.920801 0.990147 0.988946
8 0.9 0.950000 0.920759 0.989537 0.988332
9 1.0 0.949265 0.917818 0.989150 0.988946

In [5]:
nn_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in range(1, 50, 5):
    clf = MLPClassifier(hidden_layer_sizes=i)
    nn_training_pd["bc_alpha"].append(i)
    nn_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    nn_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for i in range(1, 50, 5):
    clf = MLPClassifier(hidden_layer_sizes=i)
    nn_training_pd["occ_alpha"].append(i)
    nn_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    nn_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = MLPClassifier(hidden_layer_sizes=10)
clf_occ = MLPClassifier(hidden_layer_sizes=5)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(nn_training_pd))
print(final_df)

bc_alpha bc_training bc_test occ_alpha occ_training occ_test
0 1 0.618755 0.583865 1 0.890276 0.848086
1 6 0.724084 0.838357 6 0.981167 0.969917
2 11 0.876939 0.727246 11 0.988127 0.982505
3 16 0.738704 0.755266 16 0.989355 0.969306
4 21 0.758269 0.566860 21 0.984442 0.982504
5 26 0.894459 0.368502 26 0.990174 0.950285
6 31 0.791517 0.744734 31 0.986489 0.974514
7 36 0.891645 0.876908 36 0.986285 0.966834
8 41 0.742882 0.777101 41 0.989150 0.973303
9 46 0.876769 0.620773 46 0.985875 0.976984
 percentage bc_training bc_test occ_train occ_test
0 0.1 0.651852 0.650426 0.945641 0.942886
1 0.2 0.814815 0.833291 0.957234 0.957625
2 0.3 0.432099 0.421100 0.935154 0.938588
3 0.4 0.855556 0.847570 0.955854 0.955374
4 0.5 0.829412 0.842327 0.857625 0.852201
5 0.6 0.739877 0.744629 0.931741 0.930809
6 0.7 0.681053 0.704348 0.976015 0.973388
7 0.8 0.763134 0.779838 0.904223 0.907267
8 0.9 0.616393 0.627877 0.983793 0.982600
9 1.0 0.691912 0.662361 0.961515 0.963153

In [6]:
boost_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in range(1, 100, 10):
    clf = AdaBoostClassifier(n_estimators=i, learning_rate=0.2)
    boost_training_pd["bc_alpha"].append(i)
    boost_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    boost_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for i in range(1, 100, 10):
    clf = AdaBoostClassifier(n_estimators=i, learning_rate=0.2)
    boost_training_pd["occ_alpha"].append(i)
    boost_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    boost_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = AdaBoostClassifier(n_estimators=20)
clf_occ = AdaBoostClassifier(n_estimators=20)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(boost_training_pd))
print(final_df)

bc_alpha bc_training bc_test occ_alpha occ_training occ_test
0 1 0.911978 0.934300 1 0.988946 0.985880
1 11 0.944203 0.947440 11 0.989969 0.986187
2 21 0.944203 0.942802 21 0.990379 0.988644
3 31 0.944246 0.960580 31 0.992631 0.988338
4 41 0.947229 0.960580 41 0.992835 0.989872
5 51 0.953069 0.960580 51 0.992016 0.991101
6 61 0.950171 0.956135 61 0.993040 0.989258
7 71 0.950171 0.951691 71 0.993449 0.989564
8 81 0.953069 0.951787 81 0.992835 0.990486
9 91 0.953069 0.956135 91 0.993040 0.990487
 percentage bc_training bc_test occ_train occ_test
0 0.1 1.0 0.826598 1.000000 0.985875
1 0.2 1.0 0.923657 1.000000 0.987922
2 0.3 1.0 0.932523 1.000000 0.988332
3 0.4 1.0 0.920759 1.000000 0.993654
4 0.5 1.0 0.926726 1.000000 0.992631
5 0.6 1.0 0.941347 0.999061 0.992221
6 0.7 1.0 0.912063 0.998464 0.993040
7 0.8 1.0 0.944288 0.997057 0.991402
8 0.9 1.0 0.938321 0.995735 0.991812
9 1.0 1.0 0.956010 0.994524 0.991198

In [7]:
svm_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in ["linear", "poly"]:
    clf = SVC(kernel=i)
    clf.fit(X_train_bc, y_train_bc)
    svm_training_pd["bc_alpha"].append(i)
    svm_training_pd["bc_training"].append(accuracy_score(y_train_bc, clf.predict(X_train_bc)))
    clf.fit(X_test_bc, y_test_bc)
    svm_training_pd["bc_test"].append(accuracy_score(y_test_bc, clf.predict(X_test_bc)))

for i in ["linear", "poly"]:
    clf = SVC(kernel=i)
    clf.fit(X_train_occ, y_train_occ)
    svm_training_pd["occ_alpha"].append(i)
    svm_training_pd["occ_training"].append(accuracy_score(y_train_occ, clf.predict(X_train_occ)))
    clf.fit(X_test_occ, y_test_occ)
    svm_training_pd["occ_test"].append(accuracy_score(y_test_occ, clf.predict(X_test_occ)))

clf_bc = SVC(kernel='linear')
clf_occ = SVC(kernel='linear')

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(svm_training_pd))
print(final_df)

bc_alpha bc_training bc_test occ_alpha occ_training occ_test
0 linear 0.967742 0.986842 linear 0.988127 0.983425
1 poly 0.897361 0.929825 poly 0.982190 0.978821
 percentage bc_training bc_test occ_train occ_test
0 0.1 1.000000 0.862148 0.993333 0.989969
1 0.2 0.985185 0.918073 0.993086 0.987922
2 0.3 0.977778 0.941390 0.994369 0.988127
3 0.4 0.972222 0.956095 0.991555 0.989355
4 0.5 0.966176 0.953112 0.991709 0.988536
5 0.6 0.961963 0.947315 0.991724 0.988946
6 0.7 0.967368 0.944373 0.990274 0.988536
7 0.8 0.970507 0.950256 0.988676 0.988127
8 0.9 0.970492 0.958994 0.988229 0.988127
9 1.0 0.969118 0.944331 0.988127 0.988127

In [8]:
knn_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in range(1, 100, 10):
    clf = KNeighborsClassifier(n_neighbors=i)
    knn_training_pd["bc_alpha"].append(i)
    knn_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    knn_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for i in range(1, 100, 10):
    clf = KNeighborsClassifier(n_neighbors=i)
    knn_training_pd["occ_alpha"].append(i)
    knn_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    knn_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = KNeighborsClassifier(n_neighbors=10)
clf_occ = KNeighborsClassifier(n_neighbors=11)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(knn_training_pd))
print(final_df)

bc_alpha bc_training bc_test occ_alpha occ_training occ_test
0 1 0.891560 0.943092 1 0.988946 0.988951
1 11 0.915090 0.934106 11 0.989355 0.986493
2 21 0.909165 0.929952 21 0.989560 0.986493
3 31 0.903282 0.929855 31 0.989560 0.986800
4 41 0.897485 0.929855 41 0.989560 0.985572
5 51 0.891603 0.916715 51 0.989969 0.985266
6 61 0.891603 0.903575 61 0.988332 0.984346
7 71 0.891603 0.894783 71 0.987922 0.980665
8 81 0.882822 0.877391 81 0.988127 0.980358
9 91 0.876939 0.873043 91 0.986694 0.979131
 percentage bc_training bc_test occ_train occ_test
0 0.1 0.866667 0.868159 0.992821 0.981167
1 0.2 0.888889 0.891688 0.993342 0.987103
2 0.3 0.883951 0.906223 0.995563 0.989150
3 0.4 0.901852 0.909165 0.992067 0.989560
4 0.5 0.907353 0.915047 0.992426 0.989560
5 0.6 0.907975 0.912191 0.992491 0.989355
6 0.7 0.913684 0.918031 0.991517 0.989560
7 0.8 0.924424 0.920972 0.990531 0.989355
8 0.9 0.923770 0.920929 0.990276 0.988741
9 1.0 0.922794 0.920929 0.989560 0.989355